In [1]:
# addprocs(12)  # parallel computing

In [2]:
# require("demands_adjustment_Sioux_simp_gradi")

In [16]:
# read in initial demand data

function iniDemand(trip_file, pert_fac)
    file = open(trip_file)
    demands = Dict{(Int64,Int64), Float64}()
    s = 0
    for line in eachline(file)
        if contains(line, "Origin")
            s = int(split(line)[2])
        else
            pairs = split(line, ";")
            for pair in pairs
                if !contains(pair, "\n")
                    pair_vals = split(pair, ":")
                    t, demand = int(pair_vals[1]), float(pair_vals[2])
                    #perturb the ground truth demands slightly (with perturbation factor pert_fac)
                    demands[(s,t)] = demand * pert_fac
                end
            end
        end
    end                
    close(file)
    return demands
end

iniDemand (generic function with 2 methods)

In [21]:
demands = iniDemand("../data_original/SiouxFalls_trips_simp.txt", 0.9);

In [11]:
# read in arc data

type Arc
    initNode::Int 
    termNode::Int 
    capacity::Float64
    freeflowtime::Float64
    flow::Float64
end

Arc(initNode::Int, termNode::Int, capacity::Float64, freeflowtime::Float64) = 
    Arc(initNode, termNode, capacity, freeflowtime, 0.)

function arcData(arc_file)
    arcs = Dict{(Int, Int), Arc}()
    file = open(arc_file)
    inHeader=true
    for line in eachline(file)
        if inHeader
            inHeader = !contains(line, "Init node")
            continue
        end
        vals = split(line, )
        arcs[(int(vals[1]), int(vals[2]))] = Arc(int(vals[1]), int(vals[2]), float(vals[3]), float(vals[5]))
    end
    close(file) 
    return arcs
end

arcData (generic function with 2 methods)

In [13]:
# read in observed flow data
function observFlow(arc_file, flow_file)
    arcs = arcData(arc_file)
    file = open(flow_file)
    ix = 0; 
    for line in eachline(file)
        ix +=1
        if ix ==1
            continue
        end
        vals = split(line)
        arcs[(int(vals[1]), int(vals[2]))].flow = float(vals[3])
    end
    close(file)
    return arcs
end

observFlow (generic function with 2 methods)

In [22]:
arcs = observFlow("../data_original/SiouxFalls_net_simp.txt", "../data_original/flows_converge_simp.txt");

In [24]:
# Set up demand data and flow data (for the inverse VI use)

function setUpInvVIData(demands, arcs)
    vArcs = Arc[]
    for arc in values(arcs)
        push!(vArcs, arc)
    end
    flow_data = Array(Float64, length(arcs))
    flows = Dict{(Int64,Int64), Float64}()
    demand_data = Dict{(Int, Int), Array{Float64, 1}}()
    for odpair in keys(demands)
        if ! haskey(demand_data, odpair)
            demand_data[odpair] = [demands[odpair], ]
        else
            push!(demand_data[odpair], demands[odpair])
        end
    end
    flow_data = [a.flow::Float64 for a in vArcs]
    for a in vArcs
        flows[(a.initNode, a.termNode)] = a.flow
    end
    return demand_data, flow_data, flows
end

setUpInvVIData (generic function with 1 method)

In [26]:
demands = iniDemand("../data_original/SiouxFalls_trips_simp.txt", 0.9);
arcs = observFlow("../data_original/SiouxFalls_net_simp.txt", "../data_original/flows_converge_simp.txt");

demand_data, flow_data, flows = setUpInvVIData(demands, arcs);

In [27]:
# obtain important parameters of the network

include("load_network_uni-class.jl")

function paraNetwork(demands, nameNetwork)
    ta_data = load_ta_network(nameNetwork)
    numNodes = maximum(map(pair->pair[1], keys(demands)))
    start_node = ta_data.start_node
    capacity = ta_data.capacity
    free_flow_time = ta_data.free_flow_time
    numLinks = size(start_node)[1]
    numODpairs = numNodes * (numNodes - 1)
    return numNodes, numLinks, numODpairs, capacity, free_flow_time
end

paraNetwork (generic function with 1 method)

In [30]:
numNodes, numLinks, numODpairs, capacity, free_flow_time = paraNetwork(demands, "Sioux_simp");

In [42]:
# obtain further info of the network

using JSON

function furInfo()
    
    #get number of routes
    numRoutes = readall("numRoutes.json")
    numRoutes = JSON.parse(numRoutes)

    #load OD pair-route incidence
    odPairRoute = readall("od_pair_route_incidence_Sioux_simp.json")
    odPairRoute = JSON.parse(odPairRoute)

    #load link-route incidence
    linkRoute = readall("link_route_incidence_Sioux_simp.json")
    linkRoute = JSON.parse(linkRoute)

    link_label_dict = readall("link_label_dict_Sioux_simp.json")
    link_label_dict = JSON.parse(link_label_dict)
    
    return numRoutes, odPairRoute, linkRoute, link_label_dict
end

furInfo (generic function with 1 method)

In [43]:
numRoutes, odPairRoute, linkRoute, link_label_dict = furInfo();

In [44]:
# load flows (output by TAP)
tapFlows = readall("tapFlows.json")
tapFlows = JSON.parse(tapFlows);

In [ ]:

x_0 = zeros(size(start_node))
for k = 1:length(x_0)
    key = (int(split(link_label_dict["$(k-1)"], ',')[1]), int(split(link_label_dict["$(k-1)"], ',')[2]))
    x_0[k] = flows[key]
end

In [ ]:
fcoeffs = [1, 0, 0, 0, .15]

In [ ]:
using JuMP
using Gurobi

## Obtain $\left( {\frac{{\partial {c_a}\left( {{g^l}} \right)}}{{\partial {v_a}}};a \in \mathcal{A}} \right)$ 

function sa(x, a)  # calculate the partial derivatives of c_a w.r.t. x_a
    assert(a <= length(x) && a >= 1)
    n = length(fcoeffs)
    dcdx = 0
    for i=2:n
        dcdx += (i-1) * fcoeffs[i] * (x[a]/capacity[a])^(i-2)
    end
    dcdx *= free_flow_time[a]/capacity[a]
    return dcdx
end

x = zeros(size(start_node))
for k = 1:length(x)
    key = string((int(split(link_label_dict["$(k-1)"], ',')[1]), int(split(link_label_dict["$(k-1)"], ',')[2])))
    x[k] = tapFlows[key]
end

x

saVec = similar(x)
for a = 1:length(x)
    saVec[a] = sa(x, a)
end

# solve [P2]
function solveJacob(i_th)
    assert(i_th >= 1 && i_th <= numODpairs)
    
    jacobi = Model(solver=GurobiSolver(OutputFlag=false))

    @defVar(jacobi, d[1:numLinks])
    @defVar(jacobi, x[1:numRoutes])

    for i=1:numODpairs
        sumLamX = 0
        for j=1:numRoutes
            if "$(i)-$(j)" in keys(odPairRoute)
                sumLamX += x[j]
            end
        end
        if i == i_th
            @addConstraint(jacobi, sumLamX == 1)
        else
            @addConstraint(jacobi, sumLamX == 0)
        end
    end

    for i=1:numLinks
        sumDeltaX = 0
        for j=1:numRoutes
            if "$(i)-$(j)" in keys(linkRoute)
                sumDeltaX += x[j]
            end
        end
        @addConstraint(jacobi, sumDeltaX == d[i])
    end

    @setObjective(jacobi, Min, sum{saVec[i] * (d[i])^2, i = 1:length(numLinks)})

    solve(jacobi)

    return getValue(d)
end

In [ ]:
# jacob = zeros(length(demandsVec), numLinks)
# jacob = dzeros((length(demandsVec),numLinks), workers()[1:8], [8,1])

@time y = @parallel vcat for i=1:numODpairs
    solveJacob(i)
#     d = solveJacob(i)
#     println(d)
#     for j=1:numLinks
#         jacob[i,j] = d[j]
#     end
#     println("$(i)-th row completed")
end

y = reshape(y, numODpairs, numLinks)
println(y)

In [3]:
outfile = open("jacobian.json", "w")

JSON.print(outfile, y)

close(outfile)

# M = {1:length(demandsVec)}
# pmap(solveJacob, M)

elapsed time: 42.463963846 seconds (20406753048 bytes allocated, 30.60% gc time)
[-0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 1.0 -0.0 0.0 0.0 0.0 1.0 -0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0
 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0
 -1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 